In [2]:
import sys
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

sys.version

sys.path.append("../../")

# from src.ad_hmm import sku_predict
# from src.ad_stl_model import stl_model
from src.utils import *

In [3]:
%load_ext autoreload 
%autoreload 2

### y-actual

In [4]:
original_validation = pd.read_excel("../../data/Validation_Data.xlsx").drop_duplicates()
y_act = original_validation[["Encoded_SKU_ID","SALES_DATE","DAILY_UNITS"]].rename(columns = {"DAILY_UNITS":"actual"})
y_act.head()

,Encoded_SKU_ID,SALES_DATE,actual
0,23,2022-08-06,3
1,100,2022-08-02,2
2,121,2022-08-06,6
3,207,2022-08-03,0
4,416,2022-08-05,9


###  Read all predictions

In [5]:
#y_null
y_pred_null = pd.read_csv("../../results/null_model_results_final.csv",index_col=0)
y_pred_null["SALES_DATE"] = pd.to_datetime(y_pred_null["SALES_DATE"])
y_pred_null.Encoded_SKU_ID.nunique()

539

In [7]:
#y_holt
y_pred_holt = pd.read_csv("../../results/holt_rst_final_02.csv",index_col=0)
y_pred_holt["SALES_DATE"] = pd.to_datetime(y_pred_holt["SALES_DATE"])
y_pred_holt.Encoded_SKU_ID.nunique()

41

In [10]:
#y_hmm
y_pred_hmm = pd.read_csv("../../results/hmm_result_final_all2.csv").drop(columns=["Unnamed: 0"])
y_pred_hmm["SALES_DATE"] = pd.to_datetime(y_pred_hmm["SALES_DATE"])
y_pred_hmm.Encoded_SKU_ID.nunique()

300

In [12]:
#y_prophet_stl
import pickle
with open('../../data/stl_prophet_final_results_127sku2.pkl', 'rb') as f:
    all_preds = pickle.load(f)
    
y_pred_prophet_stl_mstl = pd.DataFrame()
for i in range(len(all_preds)):
    if all_preds[i]!= None:
        y_pred_prophet_stl_mstl = pd.concat([y_pred_prophet_stl_mstl,all_preds[i][1]])
y_pred_prophet_stl_mstl = y_pred_prophet_stl_mstl.reset_index().drop(columns=["actual"])
y_pred_prophet_stl_mstl["SALES_DATE"] = pd.to_datetime(y_pred_prophet_stl_mstl["SALES_DATE"])
y_pred_prophet_stl_mstl.Encoded_SKU_ID.nunique()

Importing plotly failed. Interactive plots will not work.


219

In [14]:
y_pred_stl = y_pred_prophet_stl_mstl[["SALES_DATE","STL_prediction","Encoded_SKU_ID"\
                        ]].rename(columns={"STL_prediction":"predicted"})

y_pred_mstl = y_pred_prophet_stl_mstl[["SALES_DATE","MSTL_prediction","Encoded_SKU_ID"\
                        ]].rename(columns={"MSTL_prediction":"predicted"})

y_pred_prophet = y_pred_prophet_stl_mstl[["SALES_DATE","prophet_prediction","Encoded_SKU_ID"\
                        ]].rename(columns={"prophet_prediction":"predicted"})

### Merge all predictions

In [15]:
print("null",y_pred_null["Encoded_SKU_ID"].nunique())
print("stl",y_pred_stl["Encoded_SKU_ID"].nunique())
print("mstl",y_pred_mstl["Encoded_SKU_ID"].nunique())
print("holt",y_pred_holt["Encoded_SKU_ID"].nunique())
print("hmm",y_pred_hmm["Encoded_SKU_ID"].nunique())
print("prophet",y_pred_prophet["Encoded_SKU_ID"].nunique())

null 539
stl 219
mstl 219
holt 41
hmm 300
prophet 219


In [17]:
all_predictions = y_pred_hmm.merge(y_pred_prophet_stl_mstl,on=["Encoded_SKU_ID","SALES_DATE"],how="outer"\
                                            ).merge(y_pred_null,on=["Encoded_SKU_ID","SALES_DATE"],how="outer",\
                                                   suffixes = ("_15","_null"),\
                                                   ).merge(y_pred_holt1,on=["Encoded_SKU_ID","SALES_DATE"],how="outer"\
                                                          ,suffixes=("","_holt")).drop_duplicates()
all_predictions.head()

,SALES_DATE,predicted_15,predicted_30,predicted_45,predicted_60,predicted_90,predicted_max,Encoded_SKU_ID,STL_prediction,MSTL_prediction,prophet_prediction,predicted_null,predicted_holt
0,2022-08-01,3.00,3.00,3.00,3.00,2.00,3.00,1,NaN,NaN,NaN,1,NaN
1,2022-08-01,1.55,0.45,0.45,0.45,2.00,1.00,1,NaN,NaN,NaN,1,NaN
2,2022-08-01,1.55,1.55,1.00,2.00,2.00,2.00,1,NaN,NaN,NaN,1,NaN
3,2022-08-01,3.00,3.00,2.00,3.00,1.45,0.45,1,NaN,NaN,NaN,1,NaN
4,2022-08-01,1.00,0.00,0.00,2.00,2.00,2.00,1,NaN,NaN,NaN,1,NaN


### RMSE

In [18]:
rmse_val_null,rmse_val_models = rmse(y_act,y_pred_null),rmse(y_act,all_predictions)
rmse_val_null,rmse_val_models

(6.59603916258372, 2.2094858172197425)
